In [1]:
import numpy as np
import pandas as pd
import pysam
from Bio import pairwise2
from matplotlib import pyplot as plt
%matplotlib inline

# Methods

__Вычисление статистики коррекции ридов__ производилось следующим образом: из sam-файла доставалась последовательность рида и его выравнивание (MD-tag). Для исправленного рида и обычного находились множества позиций, в которых стоят N, и позиций с ошибками. Далее смотрелись различные пересечения/разности этих множеств и множества всех позиций в риде.

In [10]:
def compare_bamfiles(filename, filename_corr):
    stats = pd.DataFrame({'Error in corrected reads' : [0, 0],
                          'Correct base in corrected reads' : [0, 0],
                          'Base is absent in corrected reads' : [0, 0]
                         })
    stats = stats.set_index(pd.Series(['Error in raw data', 'Correct base in raw data']))
    stats_array = [0 for _ in range(6)]
    read_positions = get_read_pos_set(filename)
    
    all_reads = open(filename, 'r')
    curr_read, curr_read_rev = ["" for _ in range(4)], ["" for _ in range(4)]
    corr_rn = ''
    
    with open(filename_corr) as corr_reads:
        for read in corr_reads:
            read = read.split(' ')
            
            if read[0] != corr_rn:
                corr_rn = read[0]
                while curr_read[0] != read[0]:
                    curr_read = next(all_reads).split(' ')
                curr_read_rev = next(all_reads).split(' ')
                
                if curr_read[1].startswith('-'):
                    curr_read, curr_read_rev = curr_read_rev, curr_read
              
            
            shift = align_reads(curr_read_rev if read[1].startswith('-') else curr_read, read)
            errs_corr = count_errors(read, shift)
            
            if read[1].startswith('-'):
                update_stats(stats_array, count_errors(curr_read_rev), errs_corr, read_positions)
            else:
                update_stats(stats_array, count_errors(curr_read), errs_corr, read_positions)
            
    stats['Error in corrected reads'] = stats_array[:2]
    stats['Correct base in corrected reads'] = stats_array[2:4]
    stats['Base is absent in corrected reads'] = stats_array[4:]
    
    return stats

In [14]:
def align_reads(read, read_corr):
    if len(read[2]) == len(read_corr[2]):
        return 0
    
    read_len = len(read[2])
    shift = 0
    aligned_corr = read_corr[2]
    p1, p2 = int(read[4]), int(read_corr[4])
    
    if read[1].startswith('-') and p1 >= p2 \
       and len(aligned_corr) + (p1 - p2) <= read_len:
        shift = p1 - p2
        
    elif (not read[1].startswith('-')) and p1 <= p2\
       and len(aligned_corr) + (p2 - p1) <= read_len:
        shift = p2 - p1
        
    else:
        return 0
        
    aligned_corr =('N' * shift) + aligned_corr 
    read_corr[2] = aligned_corr + ('N' * (read_len - len(aligned_corr)))
    return shift

In [4]:
def update_stats(stats_array, errs, errs_corr, read_positions):
    stats_array[0] += len(errs_corr['err'] & errs['err'])
    stats_array[1] += len(errs_corr['err'] - errs['err'] - errs['N'])
    stats_array[2] += len(errs['err'] - errs_corr['err'])
    stats_array[3] += len((read_positions - errs['err'] - errs['N']) 
                          & (read_positions - errs_corr['err'] - errs_corr['N'])) 
    stats_array[4] += len(errs_corr['N'] & errs['err'])
    stats_array[5] += len(errs_corr['N'] - errs['err'] - errs['N'])

In [15]:
def count_errors(read, shift=0):
    errs = {'err': set(), 'N': set()}
    
    for i in range(len(read[2])):
        if read[2][i] == 'N':
            errs['N'].add(i)
    
    int_ = ''
    isdel = False
    curr_pos = 0
    for c in read[3].split(':')[2].replace('\n', ''):
        
        if c.isdigit():
            int_ += c
            isdel = False
        elif c in "ACTG" and not isdel:
            curr_pos += int(int_)
            errs['err'].add(curr_pos + shift)
            curr_pos += 1 
            int_ = ''
        elif isdel:
            continue
        elif c == '^':
            isdel = True
            curr_pos += int(int_)
            int_ = ''
        else:
            curr_pos += int(int_)
            int_ = ''
            
    return errs

In [6]:
def get_read_pos_set(filename):
    bamfile = open(filename, 'r')

    for read in bamfile:
        return set(range(len(read.split(' ')[2])))

# Procesing dataset

``bwa index MG1655-K12.first10K.fasta ``

``bwa mem MG1655-K12.first10K.fasta ecoli_10K_err_1.fastq ecoli_10K_err_2.fastq > ecoli_aln.sam``

``samtools sort -n ecoli_aln.sam > ecoli_aln.bam``

``samtools view -h ecoli_aln.bam > ecoli_aln_sorted.sam``

``cat  ecoli_aln_sorted.sam | awk 'NR>3 {print $1, $9, $10, $13}' > ecoli_aln_features``

# BayesHammer correction statistics

``spades.py -k 55 --careful --only-error-correction --pe1-1 ecoli_10K_err_1.fastq  --pe1-2 ecoli_10K_err_2.fastq -o spades_correction``

## ecoli_10K

In [40]:
stats = compare_bamfiles('data2/ecoli_aln_features', 'data2/ecoli_aln_corr_features')
stats

,Error in corrected reads,Correct base in corrected reads,Base is absent in corrected reads
Error in raw data,3099,48327,30185
Correct base in raw data,9017,5230155,515351


## ecoli_400K

In [41]:
stats = compare_bamfiles('data1/ecoli_aln_features', 'data1/ecoli_aln_corr_features')
stats

,Error in corrected reads,Correct base in corrected reads,Base is absent in corrected reads
Error in raw data,218364,2062525,1222058
Correct base in raw data,438655,249219560,19995699


# Quake correction statistics

``jellyfish count --quality-start 33  -m 15 -s 10000000 -C ecoli_10K_err_1.fastq ecoli_10K_err_2.fastq ``

``jellyfish dump mer_counts.jf > mer_counts.txt``

``parse_kmers.py > kmers.txt``

``cov_model.py --int kmers.txt ``

``correct -f fastq_files -q 33 -k 15 -c 5 -m kmers.txt``

## ecoli_10K

In [16]:
stats_q = compare_bamfiles('data2/ecoli_aln_features', 'data2/ecoli_quake_features')
stats_q

,Error in corrected reads,Correct base in corrected reads,Base is absent in corrected reads
Error in raw data,2590,45600,16393
Correct base in raw data,1284,5329863,283355


## ecoli_400K

In [17]:
stats_q = compare_bamfiles('data1/ecoli_aln_features', 'data1/ecoli_quake_features')
stats_q

,Error in corrected reads,Correct base in corrected reads,Base is absent in corrected reads
Error in raw data,162848,1993658,669123
Correct base in raw data,63841,253265878,11183960
